In [ ]:
import pan.xapi
import xml.etree.ElementTree as ET
import ipaddress
from yaml import dump
import sys

In [ ]:
options={}
with open(".pancfg") as f:
    for line in f:
        options[line.split(' ')[0].strip()] = line.split(' ')[1].strip()
if not ( options['HOSTNAME'] and options['KEY'] ):
        print (".pancfg missing or bad")
        sys.exit

In [ ]:
xapi = pan.xapi.PanXapi(api_key=options['KEY'],hostname=options['HOSTNAME'])

xapi.op(cmd='show devices connected',cmd_xml=True)

root_panorama = ET.fromstring(xapi.xml_result())

firewalls_number = len(root_panorama.findall('entry'))

In [ ]:
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

fw_num_counter = 0

output={} # nested dictionary populated with output

for firewall in root_panorama.iter(tag='entry'):

    fw_interface_no_address=False
    
    if firewall.find('serial') is not None:
        fw_serial = firewall.find('serial').text
        fw_hostname = firewall.find('hostname').text

        progress(fw_num_counter,firewalls_number,fw_hostname)
    
        fw_num_counter += 1
        
        output[fw_hostname]={}
        output[fw_hostname]["serial"]=fw_serial
        output[fw_hostname]["interfaces"]={}

        xapi_fw = pan.xapi.PanXapi(api_key=options['KEY'],hostname=options['HOSTNAME'],serial=fw_serial)
        #xapi_fw = pan.xapi.PanXapi(api_key=key,hostname=hostname,serial="001606079185")
                
        xapi_fw.op(cmd='show interface "all"',cmd_xml=True)
        fw_interfaces = ET.fromstring('<root>'+xapi_fw.xml_result()+'</root>') #adding <root> node to fix broken XML
       
        for fw_interface in fw_interfaces.iter(tag='entry'):

            if fw_interface.find('zone') is not None:
                if fw_interface.find('zone').text == 'UNTRUST-L3':
                    
                    try:
                        fw_int_ip = ipaddress.ip_interface(fw_interface.find('ip').text)
                    except Exception as exception:
                        fw_int_ip=ipaddress.ip_interface('0.0.0.0/0') # Normalizing exceptions with N/A addresses
                        fw_interface_no_address=True

                    #print(fw_int_ip.exploded)
                    
                    fw_interface_name=fw_interface.find('name').text
                    
                    if (not fw_interface_no_address) and (not fw_int_ip.is_link_local):
                        output[fw_hostname]["interfaces"][fw_interface_name]={}
                        output[fw_hostname]["interfaces"][fw_interface_name]["ip_address"]=fw_int_ip.exploded
                    
                    # if address is private start checking NAT table

                    if ( fw_int_ip.is_private ) and ( not fw_int_ip.is_link_local ) and ( not fw_int_ip.ip.compressed == '0.0.0.0' ): 

                        output[fw_hostname]["interfaces"][fw_interface_name]={}
                        output[fw_hostname]["interfaces"][fw_interface_name]["ip_address"]=fw_int_ip.exploded
                            
                        try: # get PAT rule name for this address
                            xapi_fw.op(cmd='test nat-policy-match protocol "6" source "' + \
                                fw_int_ip.ip.compressed + \
                                '" from "LAN-CORP" to "UNTRUST-L3" destination "8.8.8.8" destination-port "80"',cmd_xml=True)
                            fw_nat_rule_name = ET.fromstring('<root>' + xapi_fw.xml_result() + '</root>')
                            
                            no_nat_rule=False
                            
                            output[fw_hostname]["interfaces"][fw_interface_name]["nat_rule"]=fw_nat_rule_name[0][0].text
                         
                        except Exception as exception:
                            no_nat_rule=True
                            output[fw_hostname]["interfaces"][fw_interface_name]["nat_rule"]="No NAT rule associated"
                            
                        # collect running config for this rule, to parse <translated-address> entry
                        if not no_nat_rule:
                            xapi_fw.show(xpath="/config/devices/entry/vsys/entry/rulebase/nat/rules/entry[@name='" +\
                                fw_nat_rule_name[0][0].text +\
                                "']")
                            fw_nat_rule_config = ET.fromstring('<root>' + xapi_fw.xml_result() + '</root>')
                            fw_nat_rule_tr_ad = fw_nat_rule_config.find('entry/source-translation/dynamic-ip-and-port/translated-address/member').text
                        
                            output[fw_hostname]["interfaces"][fw_interface_name]["translated_address"] = fw_nat_rule_tr_ad
                            
                            # collect IP address of translated object address
                            xapi_fw.show(xpath="/config/devices/entry/vsys/entry/address/entry[@name='" +\
                                fw_nat_rule_tr_ad +\
                                "']")
                            fw_ip_obj_name = ET.fromstring('<root>' + xapi_fw.xml_result() + '</root>')

                            output[fw_hostname]["interfaces"][fw_interface_name]["translated_ip"] = fw_ip_obj_name[0][0].text
    #break
print ("\nDone")

In [ ]:
from yaml import dump
print (yaml.dump(output,default_flow_style=False))

In [ ]:
with open("ext_interfaces_list", "w") as f:
    f.write (yaml.dump(output,default_flow_style=False))